In [ ]:
# Set random seeds
from IPython.display import display, HTML
from numpy.random import seed
seed(156)
import tensorflow as tf
tf.random.set_seed(256)

In [ ]:
import sys
print(sys.version)

3.7.15 (default, Oct 12 2022, 19:14:55) 
[GCC 7.5.0]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab_Notebooks/safety_report_tc

/content/drive/MyDrive/Colab_Notebooks/safety_report_tc


In [ ]:
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 287 kB 6.6 MB/s 
     |████████████████████████████████| 106 kB 63.2 MB/s 


In [ ]:
# Importing required libraries
import pandas as pd
import re
import requests
import nltk
import inflect
import contractions
from bs4 import BeautifulSoup
import re, string, unicodedata
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from tqdm.std import tqdm
from IPython.display import display, HTML
import time
import textwrap
import requests
import json

In [ ]:
# Install required nltk resources
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Pick a minority category of interest (base data from source)
focus_cats = [
    'hydraulic fluid or oil leak',
    'line strike',
    'site compliance or practice issue',
    'ppe non-compliance',
    'mechanical or equipment issue'
    ]

# Base case data used for training, validation and testing
fns = [
    'hydraulic fluid or oil leak_577_out_df_temp',
    'line strike_326_out_df_temp',
    'site compliance or practice issue_400_out_df_temp',
    'ppe non-compliance_144_out_df_temp',
    'mechanical or equipment issue_471_out_df_temp'
]

# Additional data recieved after models were created and tested
fnt = [
    'hydraulic fluid or oil leak_31_out_df_temp',
    'line strike_8_out_df_temp',
    'site compliance or practice issue_6_out_df_temp',
    'ppe non-compliance_7_out_df_temp',
    'mechanical or equipment issue_11_out_df_temp'
]

# NOTE USED IN FINAL SOLUTION
# Filename for fabricated data(by decoder) file
# This is used at the end for denoising & normalising the fabricated data before
# use in TC modelling.
fnfs = 'hydraulic fluid or oil leak_7920'

In [ ]:
# Select a category to load
pick = input('Pick a category: ')
data = input('Original data (o) or new (n): ')
index = focus_cats.index(pick)
focus_cat = focus_cats[index]

if data == 'o':
    fn = fns[index]
elif data == 'n':
    fn = fnt[index]

fnf = fnfs[index]

# Load the base data
df = pd.read_csv(f'08_output/{fn}.csv')
df = df[['text', 'category']]

# Convert category to binary 1/0
# Remeber that 1 is a rule-book hit & not necessarily the 'truth'
df['category'] = df['category'].apply(lambda x : x.replace("*** Not Classified", "0"))
df['category'] = df['category'].apply(lambda x : x.replace(focus_cat, "1"))
df['category'] = df['category'].astype(int)
df

Pick a category: mechanical or equipment issue
Original data (o) or new (n): n


,text,category
0,person tested positive to covid 19 in camp per...,0
1,scaffolder struck thumb with hammer causing mi...,0
2,ip reported eye irritation whilst working on t...,0
3,wood employee tested antigen rapid test (art) ...,0
4,one (1) massy wood employee tested positive fo...,0
...,...,...
2424,a managed contractor employee tested positive ...,0
2425,cp-2022-0043 env nanjv; release of silt laden ...,0
2426,cut to knee when ascending staircase individua...,0
2427,one wood employee tested positive for covid-19...,0


Inspired by
https://github.com/pashupati98/text-classification

## Data Preparation

In [ ]:
STOP_SENTS = ['migrated from legacy cairs', 'migrated from cairs']

f = open('01_data/us2gb.json')
US2GB = json.load(f)
f.close()

def remove_stop_sentences(in_doc):
    for i, sen in enumerate(STOP_SENTS):
        if i==0:
            rep_doc = in_doc
        else:
            rep_doc = out_doc
        out_doc = rep_doc.replace(f'{sen}', '') 
    out_doc = re.sub(' +', ' ', out_doc)    
    return(out_doc)

def americanize(in_doc):
    for american_spelling, british_spelling in US2GB.items(): 
        in_doc = re.sub(f'(?<![a-zA-Z]){british_spelling}(?![a-z-Z])', american_spelling, in_doc)
    return(in_doc)

# Main denoising function
def denoise_text(txt):
    # Strip html if any.
    soup = BeautifulSoup(txt, "html.parser")
    txt = soup.get_text()

    # Replace contractions in string of text. For ex. didn't -> did not
    txt = contractions.fix(txt)

    # Change to lower case
    txt = txt.lower()

    # Remove any defined stop sentences
    txt = remove_stop_sentences(txt)

    # Convert British to American spelling
    #txt = americanize(txt)
    return txt

In [ ]:
# Test the denoising function
chk_text = "<p>she didn't TELL me anything </br> about what's gonna <html> happen in the end but did want to optimise the colours migrated from cairs"
denoise_text(chk_text)

'she did not tell me anything about what is going to happen in the end but did want to optimise the colours '

In [ ]:
# Normalization may include everal step
# Each function below fulfills a (potential) step in normalization

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        split_words = word.split('.')
        for word in split_words:
            new_word = re.sub(r'[^\w\s]', '', word)
            if new_word != '':
                new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

# Define the normalization pipeline
# Comment out steps not used
def normalize_text(words, lema=False, stem=False, stop=False):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    # TODO - Experiment with the below options
    # Inuitively, stopwords should not be removed as they may impact
    # semantic meaning.
    if stop: words = remove_stopwords(words) 
    if stem: words = stem_words(words)
    if lema: words = lemmatize_verbs(words)
    
    return words


In [ ]:
# Let's test the individual normalization functions
print("Remove non_ascii: ", remove_non_ascii(['h', 'ॐ', '©', '1']))
print("To lowercase: ", to_lowercase(['HELLO', 'hiDDen', 'wanT', 'GOING']))
print("Remove punctuation: ", remove_punctuation(['hello!!', 'how?', 'done,', 'hello.you', 'You my friend.Are a fool.']))
print("Replace numbers: ", replace_numbers(['1', '2', '3']))
print("Remove stopwords: ", remove_stopwords(['this', 'and', 'amazing', 'not', 'no', 'yes']))
print("Stem words: ", stem_words(['beautiful', 'flying', 'waited']))
print("Lemmatize verbs: ", lemmatize_verbs(['hidden', 'walking', 'ran']))

# Now apply the pipeline (on a test)
print("Normalize text: ", normalize_text(['hidden', 'in', 'the', 'ALMIRAH', 'he', 'WAited', '2', 'ॐ', 'hours!!']))

Remove non_ascii:  ['h', '', '', '1']
To lowercase:  ['hello', 'hidden', 'want', 'going']
Remove punctuation:  ['hello', 'how', 'done', 'hello', 'you', 'You my friend', 'Are a fool']
Replace numbers:  ['one', 'two', 'three']
Remove stopwords:  ['amazing', 'yes']
Stem words:  ['beauty', 'fly', 'wait']
Lemmatize verbs:  ['hide', 'walk', 'run']
Normalize text:  ['hidden', 'in', 'the', 'almirah', 'he', 'waited', 'two', 'hours']


In [ ]:
# Tokenize text into words
def simple_tokenize(text):
    return nltk.word_tokenize(text)

# Check the function
sample_text = 'he did not say anything  about what is going to  happen'
print("tokenize results :", simple_tokenize(sample_text))

tokenize results : ['he', 'did', 'not', 'say', 'anything', 'about', 'what', 'is', 'going', 'to', 'happen']


In [ ]:
# Now create a simple function to denoise, normalize and apply simple tokeniser to text
def text_prepare(text):
    text = denoise_text(text)
    text = ' '.join([x for x in normalize_text(simple_tokenize(text))])
    return text

In [ ]:
re.sub(" s ", " ", text_prepare("I am not gonna go to the shop's to visualise"))

'i am not going to go to the shop to visualise'

In [ ]:
# Now apply the main text prep pipeline to all text:
# Takes about 5 mins to run on Google CoLab
df['text'] = [text_prepare(x) for x in tqdm(df['text'])]

# Clean up on category encoding
le = LabelEncoder()
df['category'] = le.fit_transform(df['category'])
print('Done!')
df.head()

100%|██████████| 2429/2429 [00:03<00:00, 809.25it/s]

Done!


,text,category
0,person tested positive to covid nineteen in ca...,0
1,scaffolder struck thumb with hammer causing mi...,0
2,ip reported eye irritation whilst working on t...,0
3,wood employee tested antigen rapid test art po...,0
4,one one massy wood employee tested positive fo...,0


In [ ]:
df['text'] = df['text'].apply(lambda x: re.sub(' s ', ' ', x))
df.head()

,text,category
0,person tested positive to covid nineteen in ca...,0
1,scaffolder struck thumb with hammer causing mi...,0
2,ip reported eye irritation whilst working on t...,0
3,wood employee tested antigen rapid test art po...,0
4,one one massy wood employee tested positive fo...,0


In [ ]:
table = df[df['category']==1]
display(HTML(table.to_html()))

,text,category
100,during mechanical excavation for the installation of the vdrain at the igf area the skid steer damaged the plastic cover of a cable on the day of the incident a skid steer was busy with mechanical excavation for the vdrain in the cpf igf area the area had been cleared for mechanical excavation and an excavation permit was available the cable in question is for an old street light circuit and was identified by the work crew the work crew assumed that the cable was running all the way along the pipe rack and proceeded with the excavation as planned the cable was exposed for about two meters and then deviated in the area where it was damaged to the inside path of the excavation the operator of the skid steer placed the bucket and was stopped by the spotter when he saw the cable the plastic cover was damaged but no damage occurred to the armoring of the cable the area was made safe damage was assessed by the client electrical team the following day and classified as minorlight circuit was disconnected as it is no longer in use and part of an old installation client and wood project teams were notifiedincident investigation planned for the 01st november two thousand and twenty-two,1
149,cat seven hundred and seventy-seven haul truck turned front right bumper into rock wall while backing up resulting in bending bumper on october ten two thousand and twenty-two a cat seven hundred and seventy-seven was driving into the dpit dump area to unload the truck entered the pit on the left roadway turned right up a small ramp stopped and backed to the left into the dumping area while negotiating the backward left turn the operator got too close to the wall with the right blind side of the truck the right bumper contacted the wall knocking a large rock out and bending the bumper up the operator did not realize they had struck the wall until the next driver asked about the rock in the roadway they stopped the truck to do an inspection and found the damaged bumper a stop work in the dpit was called and the haul truck was parked to conduct an investigation,1
190,employees were predrilling holes for piles when a hydraulic hose ruptured causing a fluid spill subcontract employees were predrilling holes for piles when a hydraulic hose ruptured causing a spill the equipment being operated was a comacchio hitech mch drill rig the hydraulic hose had been rubbing on a piece of the equipment frame which caused the rupture the hydraulic leak occurred twenty feet away from personnel working in the area approximately four gallons fifteen fourteen liters of hydraulic fluid leaked onto the ground and was contained with a spill kit both the potential and the actual were selected as a1 as there was no personnel at risk in the incident a spill kit was immediately utilized to prevent the spread of the fluid all contaminated ground was removed and bagged for environmental disposal a service technician onsite removed the damaged hose and capped the connections to prevent further fluid loss an investigation was initiated,1
291,wood parked vehicle on site was bumped into causing damage to the break light qc inspector had parked near t4228 to complete an inspection with the client the van was parked on a road which is not a through road as one end was blocked off with a barrier there was three other vehicles parked on the same road one was another wood vehicle driven by a wood site supervisor and was parked behind the damaged vehicle one was the standby mans vehicle which was not parked anywhere near the damaged vehicle and the last vehicle was another business partner of essar the qc inspector entered the tank and on return approx twenty minutes later he found the damage on the vehicle reported to the hhse advisor and vehicle returned to workshop to be inspected statement taken from qc inspector,1
417,worker cut energized electrical cable workers were tasked with identifying and removing obsolete cable from xlr between columns g5 and g6 ground floo

In [ ]:
# Save the prepared dataset
filename = f'01_data/prepared/{time.strftime("%y%m%d%H%M")}_prepared_{focus_cat}_data.csv'
df.to_csv(filename, index=False)

### Fabricated Data Prep - NOTE USED IN FINAL SOLUTION

In [ ]:
# Load fabricated reports
df_fab = pd.read_csv(f'01_data/fabricated/{fnf}.csv')
df_fab

,text,group
0,there was a leak of oil when hydraulic fluid e...,hydraulic fluid or oil leak
1,there was a leak from a hose on a pump truck w...,hydraulic fluid or oil leak
2,there was a leak to grade one hundred and fift...,hydraulic fluid or oil leak
3,there was a leak of hydraulic fluid from a ren...,hydraulic fluid or oil leak
4,there was a leak from a jlg cat three thousand...,hydraulic fluid or oil leak
...,...,...
7915,oil fluid witnessed falling object resulting i...,hydraulic fluid or oil leak
7916,oil fluid witnessed on three out of four an em...,hydraulic fluid or oil leak
7917,oil fluid witnessed nearmisses at separation t...,hydraulic fluid or oil leak
7918,oil fluid witnessed a hydraulic leak from a na...,hydraulic fluid or oil leak


In [ ]:
df_fab['text'] = [text_prepare(x) for x in tqdm(df_fab['text'])]

100%|██████████| 7920/7920 [00:09<00:00, 846.80it/s]


In [ ]:
df_fab = df_fab.sample(frac=1).reset_index(drop=True)
df_fab

,text,group
0,crew noticed drip coming from hose fitting on ...,hydraulic fluid or oil leak
1,diesel failure on skid steer resulted in 1l of...,hydraulic fluid or oil leak
2,operative witnessed discharge of hydraulic flu...,hydraulic fluid or oil leak
3,subcontractor witnessed loss of hydraulic flui...,hydraulic fluid or oil leak
4,operative discovered leak in hose on roller dr...,hydraulic fluid or oil leak
...,...,...
7915,there was a flow of oil from the gully to the ...,hydraulic fluid or oil leak
7916,hyd fluid failure of excavator hydraulic fitti...,hydraulic fluid or oil leak
7917,apprentice noticed spillage of hydraulic fluid...,hydraulic fluid or oil leak
7918,apprentice witnessed drip from a bucket lift i...,hydraulic fluid or oil leak


In [ ]:
# Save the prepared fabricated dataset
filename = f'01_data/prepared/{time.strftime("%y%m%d%H%M")}_prepared_fabricated_{focus_cat}_data.csv'
df_fab.to_csv(filename, index=False)